In [ ]:
# Data = https://drive.google.com/file/d/1mISjAmQeC5nqo6yq_gETfEFXf9kzLnCW/view?usp=sharing

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from matplotlib.colors import Normalize
import imageio
import os

In [ ]:
data = pd.read_csv('Data_2015.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2266 entries, 0 to 2265
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   YYYYMMDD  2042 non-null   object 
 1   Location  2042 non-null   object 
 2   Country   2038 non-null   object 
 3   Topic     2042 non-null   object 
 4   Count     2042 non-null   float64
dtypes: float64(1), object(4)
memory usage: 88.6+ KB


In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
print(world)

<ipython-input-5-55e8c1fd3a05>:1: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


         pop_est      continent                      name iso_a3  gdp_md_est  \
0       889953.0        Oceania                      Fiji    FJI        5496   
1     58005463.0         Africa                  Tanzania    TZA       63177   
2       603253.0         Africa                 W. Sahara    ESH         907   
3     37589262.0  North America                    Canada    CAN     1736425   
4    328239523.0  North America  United States of America    USA    21433226   
..           ...            ...                       ...    ...         ...   
172    6944975.0         Europe                    Serbia    SRB       51475   
173     622137.0         Europe                Montenegro    MNE        5542   
174    1794248.0         Europe                    Kosovo    -99        7926   
175    1394973.0  North America       Trinidad and Tobago    TTO       24269   
176   11062113.0         Africa                  S. Sudan    SSD       11998   

                                       

In [ ]:
data['YYYYMMDD'] = pd.to_datetime(data['YYYYMMDD'])
data_grouped = data.groupby(['YYYYMMDD', 'Country']).sum().reset_index()


In [ ]:
norm = Normalize(vmin=data_grouped['Count'].min(), vmax=data_grouped['Count'].max())

In [17]:
output_dir = '/mnt/data/'
os.makedirs(output_dir, exist_ok=True)

In [18]:
filenames = []
for date in data_grouped['YYYYMMDD'].unique():
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    world.boundary.plot(ax=ax)

    data_date = data_grouped[data_grouped['YYYYMMDD'] == date]

    merged = world.set_index('name').join(data_date.set_index('Country'))

    merged.plot(column='Count', ax=ax, legend=True, cmap='autumn', norm=norm, missing_kwds={'color': 'lightgrey'})
    plt.title(f'Global Event Count on {date.date()}', fontsize=20)
    plt.axis('off')

    filename = f'{output_dir}heatmap_{date.date()}.png'
    filenames.append(filename)
    plt.savefig(filename)
    plt.close()



In [19]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [20]:
with imageio.get_writer('/mnt/data/global_event_heatmap.gif', mode='I', duration=1) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

<ipython-input-20-ef778b0b1de4>:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
